In [ ]:
# ============================================
# Kaggle-ready: Fetch + Clean (no leakage) + Build Tabular/Seq
# SINGLE KEY SET (no duplicate target keys)
# - Canonical keys: temp, rain, u10, v10, rh, press, cloud
# - Files named by LOCATION_ID (UUID) for consistency
# - 20 PROVINCES/CITIES OF VIETNAM (optimized for Kaggle free tier)
# ============================================

!pip -q install -U pyarrow requests

import os, gc, json, time
from pathlib import Path
from uuid import uuid5, NAMESPACE_DNS

import numpy as np
import pandas as pd
import requests

# -------------------------
# 1) Paths / settings (optimized for Kaggle)
# -------------------------
BASE_DIR = Path("/kaggle/working/weather_20loc")
RAW_DIR  = BASE_DIR / "data" / "raw"
PROC_DIR = BASE_DIR / "data" / "processed"
TAB_DIR  = BASE_DIR / "data" / "tabular"
SEQ_DIR  = BASE_DIR / "data" / "sequences"
META_DIR = BASE_DIR / "data" / "meta"
for p in [RAW_DIR, PROC_DIR, TAB_DIR, SEQ_DIR, META_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# -------------------------
# 2) 20 PROVINCES/CITIES OF VIETNAM (đại diện đủ các vùng miền)
# -------------------------
LOCATIONS = [
    # ===== MIỀN BẮC (7 tỉnh) =====
    {"name": "hanoi",        "lat": 21.0285, "lon": 105.8542},  # 1. TP Hà Nội
    {"name": "haiphong",     "lat": 20.8449, "lon": 106.6881},  # 2. TP Hải Phòng
    {"name": "quangninh",    "lat": 21.0064, "lon": 107.2925},  # 3. Quảng Ninh
    {"name": "laocai",       "lat": 22.4856, "lon": 103.9707},  # 4. Lào Cai
    {"name": "hagiang",      "lat": 22.8233, "lon": 104.9833},  # 5. Hà Giang
    {"name": "thainguyen",   "lat": 21.5928, "lon": 105.8442},  # 6. Thái Nguyên
    {"name": "thanhhoa",     "lat": 19.8066, "lon": 105.7852},  # 7. Thanh Hóa
    
    # ===== MIỀN TRUNG (7 tỉnh) =====
    {"name": "nghean",       "lat": 19.2342, "lon": 104.9200},  # 8. Nghệ An
    {"name": "hatinh",       "lat": 18.3559, "lon": 105.8877},  # 9. Hà Tĩnh
    {"name": "hue",          "lat": 16.4674, "lon": 107.5905},  # 10. Thừa Thiên Huế
    {"name": "danang",       "lat": 16.0544, "lon": 108.2022},  # 11. TP Đà Nẵng
    {"name": "quangnam",     "lat": 15.5735, "lon": 108.4740},  # 12. Quảng Nam
    {"name": "khanhhoa",     "lat": 12.2585, "lon": 109.0526},  # 13. Khánh Hòa
    {"name": "ninhthuan",    "lat": 11.5752, "lon": 108.9829},  # 14. Ninh Thuận
    
    # ===== TÂY NGUYÊN + NAM BỘ (6 tỉnh) =====
    {"name": "gialai",       "lat": 13.9832, "lon": 108.0025},  # 15. Gia Lai
    {"name": "daklak",       "lat": 12.6800, "lon": 108.0378},  # 16. Đắk Lắk
    {"name": "lamdong",      "lat": 11.9404, "lon": 108.4583},  # 17. Lâm Đồng
    {"name": "hcmc",         "lat": 10.8231, "lon": 106.6297},  # 18. TP Hồ Chí Minh
    {"name": "dongnai",      "lat": 10.9453, "lon": 106.8243},  # 19. Đồng Nai
    {"name": "cantho",       "lat": 10.0452, "lon": 105.7469},  # 20. TP Cần Thơ
]

def make_ids(name, lat, lon):
    grid_id = f"{lat:.4f}_{lon:.4f}"
    location_id = str(uuid5(NAMESPACE_DNS, f"loc::{name}::{grid_id}"))
    return location_id, grid_id

for loc in LOCATIONS:
    loc["location_id"], loc["grid_id"] = make_ids(loc["name"], loc["lat"], loc["lon"])

# Tạo mapping và lưu ra file để training notebooks sử dụng
LOCATION_IDS = [loc["location_id"] for loc in LOCATIONS]
LOC_META = {loc["location_id"]: {"name": loc["name"], "lat": loc["lat"], "lon": loc["lon"], "grid_id": loc["grid_id"]} for loc in LOCATIONS}

# Lưu metadata
meta_path = META_DIR / "locations.json"
with open(meta_path, "w") as f:
    json.dump({"locations": LOCATIONS, "location_ids": LOCATION_IDS}, f, indent=2)
print(f"[ok] Saved location metadata: {meta_path}")
print(f"[info] Total locations: {len(LOCATIONS)}")

for loc in LOCATIONS:
    print(f"  {loc['name']:15s} -> {loc['location_id']}")

# -------------------------
# 3) Time ranges / splits (optimized: 3 năm train, 1 năm val, 11 tháng test)
# -------------------------
FULL_START = "2021-01-01"
FULL_END   = "2025-11-30"

SPLITS = {
    "train_2021_2023": ("2021-01-01", "2023-12-31"),
    "val_2024":        ("2024-01-01", "2024-12-31"),
    "test_2025_01_to_2025_11": ("2025-01-01", "2025-11-30"),
}

# -------------------------
# 4) HYPERPARAMETERS
# -------------------------
LAG = 49            # 49h lookback
HORIZON = 100       # 100h forecast (~4 days)

# -------------------------
# 5) Open-Meteo archive fetch (với rate limit handling tốt hơn)
# -------------------------
ARCHIVE_URL = "https://archive-api.open-meteo.com/v1/archive"

HOURLY_VARS = [
    "temperature_2m",
    "relative_humidity_2m",
    "precipitation",
    "wind_speed_10m",
    "wind_direction_10m",
    "surface_pressure",
    "cloud_cover",
]

def _safe_get(url, params, tries=5, backoff=2.0, timeout=60):
    """Safe GET with exponential backoff, đặc biệt cho 429 errors"""
    last = None
    for t in range(tries):
        try:
            r = requests.get(url, params=params, timeout=timeout)
            if r.status_code == 429:
                # Rate limited - wait longer
                wait_time = backoff ** (t + 2)  # Longer wait for rate limit
                print(f"[rate limit] waiting {wait_time:.1f}s...", end=" ", flush=True)
                time.sleep(wait_time)
                continue
            r.raise_for_status()
            return r
        except Exception as e:
            last = e
            wait_time = backoff ** t
            time.sleep(wait_time)
    raise RuntimeError(f"Request failed after {tries} tries: {last}")

def fetch_openmeteo_hourly_archive(loc, start_date, end_date):
    params = {
        "latitude": loc["lat"],
        "longitude": loc["lon"],
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ",".join(HOURLY_VARS),
        "timezone": "UTC",
        "wind_speed_unit": "ms",
        "precipitation_unit": "mm",
        "timeformat": "iso8601",
    }
    r = _safe_get(ARCHIVE_URL, params=params)
    js = r.json()
    hourly = js.get("hourly", {})
    out = pd.DataFrame({"time": hourly.get("time", [])})
    for v in HOURLY_VARS:
        out[v] = hourly.get(v, None)
    out["location_id"] = loc["location_id"]
    out["grid_id"] = loc["grid_id"]
    return out

def fetch_full_to_raw():
    total = len(LOCATIONS)
    for i, loc in enumerate(LOCATIONS, 1):
        # File đặt tên theo location_id
        raw_path = RAW_DIR / f"{loc['location_id']}_hourly_raw.parquet"
        if raw_path.exists():
            print(f"[skip {i}/{total}] raw exists:", raw_path.name)
            continue
        print(f"[fetch {i}/{total}] {loc['name']}...", end=" ", flush=True)
        df = fetch_openmeteo_hourly_archive(loc, FULL_START, FULL_END)
        df.to_parquet(raw_path, index=False)
        print(f"-> {raw_path.name}", df.shape)
        # Delay để tránh rate limit
        time.sleep(1.5)

# -------------------------
# 6) Clean (NO leakage)
# -------------------------
def wind_to_uv(speed_ms, dir_deg):
    sp = pd.to_numeric(speed_ms, errors="coerce").astype("float32")
    dd = pd.to_numeric(dir_deg, errors="coerce").astype("float32")
    rad = np.deg2rad(dd)
    u = -sp * np.sin(rad)
    v = -sp * np.cos(rad)
    return u.astype("float32"), v.astype("float32")

def uv_to_wind_dir(u, v):
    u = pd.to_numeric(u, errors="coerce").astype("float32")
    v = pd.to_numeric(v, errors="coerce").astype("float32")
    deg = (np.degrees(np.arctan2(-u, -v)) + 360.0) % 360.0
    return deg.astype("float32")

def add_time_feats(df, time_col="time"):
    t = pd.to_datetime(df[time_col], utc=True)
    hour = t.dt.hour.astype(np.float32)
    doy  = t.dt.dayofyear.astype(np.float32)
    df["hour_sin"] = np.sin(2*np.pi*hour/24.0).astype("float32")
    df["hour_cos"] = np.cos(2*np.pi*hour/24.0).astype("float32")
    df["doy_sin"]  = np.sin(2*np.pi*doy/365.25).astype("float32")
    df["doy_cos"]  = np.cos(2*np.pi*doy/365.25).astype("float32")
    return df

def clean_hourly_df(df, time_col="time", freq="1h", max_interp_gap_hours=6, max_ffill_gap_hours=6, clip_precip_nonneg=True):
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col], utc=True, errors="coerce")
    df = df.dropna(subset=[time_col]).sort_values(time_col).reset_index(drop=True)

    loc_id = df["location_id"].iloc[0] if "location_id" in df.columns and len(df) else None
    grid_id = df["grid_id"].iloc[0] if "grid_id" in df.columns and len(df) else None

    for c in HOURLY_VARS:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32")

    df["u10"], df["v10"] = wind_to_uv(df["wind_speed_10m"], df["wind_direction_10m"])
    df = df.set_index(time_col)

    start = df.index.min().floor(freq)
    end   = df.index.max().ceil(freq)
    full_index = pd.date_range(start=start, end=end, freq=freq, tz="UTC")
    df = df.reindex(full_index)

    lim = int(max_interp_gap_hours)
    ffill_lim = int(max_ffill_gap_hours)

    # Continuous cols: interpolate + ffill for small gaps
    cont_cols = ["temperature_2m","relative_humidity_2m","u10","v10","surface_pressure","cloud_cover"]
    for c in cont_cols:
        s = df[c]
        s = s.interpolate(method="time", limit=lim, limit_direction="forward")
        s = s.ffill(limit=ffill_lim)
        df[c] = s.astype("float32")

    # Precipitation: fill 0 for small gaps
    precip = df["precipitation"]
    precip = precip.ffill(limit=ffill_lim)
    precip = precip.fillna(0.0)
    df["precipitation"] = precip.astype("float32")

    df["wind_speed_10m"] = np.sqrt(df["u10"]**2 + df["v10"]**2).astype("float32")
    df["wind_direction_10m"] = uv_to_wind_dir(df["u10"], df["v10"]).astype("float32")

    if clip_precip_nonneg:
        df["precipitation"] = np.maximum(df["precipitation"].to_numpy(), 0.0).astype("float32")

    df = df.reset_index().rename(columns={"index": time_col})
    df = add_time_feats(df, time_col=time_col)

    if loc_id is not None:
        df["location_id"] = loc_id
    if grid_id is not None:
        df["grid_id"] = grid_id

    return df

def process_full_raw_to_processed():
    total = len(LOCATIONS)
    for i, loc in enumerate(LOCATIONS, 1):
        raw_path  = RAW_DIR  / f"{loc['location_id']}_hourly_raw.parquet"
        proc_path = PROC_DIR / f"{loc['location_id']}_hourly_proc.parquet"
        if proc_path.exists():
            print(f"[skip {i}/{total}] proc exists:", proc_path.name)
            continue
        df = pd.read_parquet(raw_path)
        if "location_id" not in df.columns:
            df["location_id"] = loc["location_id"]
        if "grid_id" not in df.columns:
            df["grid_id"] = loc["grid_id"]
        df = clean_hourly_df(df)
        df.to_parquet(proc_path, index=False)
        print(f"[ok {i}/{total}] processed {loc['name']} -> {proc_path.name}", df.shape)
        gc.collect()

# -------------------------
# 7) Build tabular lagged + sequences (OPTIMIZED)
# -------------------------
FEATURE_COLS = [
    "temperature_2m","relative_humidity_2m","precipitation",
    "u10","v10","wind_speed_10m",
    "surface_pressure","cloud_cover",
    "hour_sin","hour_cos","doy_sin","doy_cos",
]

TAB_TARGETS = {
    "temp":  "temperature_2m",
    "rain":  "precipitation",
    "u10":   "u10",
    "v10":   "v10",
    "rh":    "relative_humidity_2m",
    "press": "surface_pressure",
    "cloud": "cloud_cover",
}

GRU_INPUT_COLS  = FEATURE_COLS
GRU_TARGET_COLS = [
    "temperature_2m","precipitation","u10","v10",
    "relative_humidity_2m","surface_pressure","cloud_cover"
]

def build_X_lags(d, time_col="time"):
    """Build lagged features efficiently using numpy"""
    data = d[FEATURE_COLS].to_numpy(dtype=np.float32)
    n_samples = len(d)
    n_features = len(FEATURE_COLS)
    
    # Pre-allocate array for lagged features
    lag_data = np.empty((n_samples, LAG * n_features), dtype=np.float32)
    lag_cols = []
    
    for l in range(LAG):
        start_col = l * n_features
        end_col = start_col + n_features
        if l == 0:
            lag_data[:, start_col:end_col] = data
        else:
            lag_data[l:, start_col:end_col] = data[:-l]
            lag_data[:l, start_col:end_col] = np.nan
        
        lag_cols.extend([f"{c}_lag{l:02d}" for c in FEATURE_COLS])
    
    X_lags = pd.DataFrame(lag_data, columns=lag_cols)
    
    keep_meta = [time_col]
    for meta in ["location_id","grid_id"]:
        if meta in d.columns:
            keep_meta.append(meta)
    return d[keep_meta].reset_index(drop=True), X_lags

def make_tabular_lagged_from_precomputed(meta_df, X_lags, d, target_col):
    """Build tabular data with future targets efficiently"""
    data = d[target_col].to_numpy(dtype=np.float32)
    n_samples = len(d)
    
    # Pre-allocate array for future targets
    y_data = np.empty((n_samples, HORIZON), dtype=np.float32)
    y_cols = []
    
    for h in range(1, HORIZON + 1):
        if h < n_samples:
            y_data[:-h, h-1] = data[h:]
            y_data[-h:, h-1] = np.nan
        else:
            y_data[:, h-1] = np.nan
        y_cols.append(f"y_t+{h:03d}")
    
    Y = pd.DataFrame(y_data, columns=y_cols)
    out = pd.concat([meta_df.reset_index(drop=True), X_lags.reset_index(drop=True), Y], axis=1)
    out = out.dropna().reset_index(drop=True)
    return out

def make_rnn_sequences(df, time_col="time"):
    """Build sequences efficiently using sliding window view"""
    d = df.copy()
    d[time_col] = pd.to_datetime(d[time_col], utc=True, errors="coerce")
    d = d.dropna(subset=[time_col]).sort_values(time_col).reset_index(drop=True)

    X = d[GRU_INPUT_COLS].to_numpy(np.float32)
    Y = d[GRU_TARGET_COLS].to_numpy(np.float32)
    t = d[time_col].to_numpy("datetime64[ns]")
    loc_id = d["location_id"].iloc[0] if "location_id" in d.columns else ""

    n = len(d)
    n_samples = n - LAG - HORIZON + 1
    if n_samples <= 0:
        raise ValueError(f"Not enough rows: n={n}, need at least {LAG+HORIZON}")

    from numpy.lib.stride_tricks import sliding_window_view

    Xw = sliding_window_view(X, window_shape=(LAG, X.shape[1]))
    Xw = Xw[:, 0, :, :]
    Xs = Xw[:n_samples].astype(np.float32)

    Yw = sliding_window_view(Y[LAG:], window_shape=(HORIZON, Y.shape[1]))
    Yw = Yw[:, 0, :, :]
    Ys = Yw[:n_samples].astype(np.float32)

    Ts = t[LAG-1 : LAG-1 + n_samples]
    meta = {"lag": LAG, "horizon": HORIZON, "x_cols": GRU_INPUT_COLS, "y_cols": GRU_TARGET_COLS, "location_id": loc_id}
    return Xs, Ys, Ts, meta

def build_all_outputs():
    """Build tabular and sequence outputs with memory optimization"""
    total = len(LOCATIONS)
    for i, loc in enumerate(LOCATIONS, 1):
        loc_id = loc["location_id"]
        proc_path = PROC_DIR / f"{loc_id}_hourly_proc.parquet"
        
        # Check if all outputs already exist
        all_exist = True
        for key in TAB_TARGETS:
            for split_name in SPLITS:
                tab_path = TAB_DIR / f"{loc_id}_{split_name}_tab_{key}_lag{LAG}_h{HORIZON}.parquet"
                if not tab_path.exists():
                    all_exist = False
                    break
            if not all_exist:
                break
        for split_name in SPLITS:
            seq_path = SEQ_DIR / f"{loc_id}_{split_name}_seq_multi_lag{LAG}_h{HORIZON}.npz"
            if not seq_path.exists():
                all_exist = False
                break
        
        if all_exist:
            print(f"[skip {i}/{total}] {loc['name']} outputs exist")
            continue
            
        print(f"[build {i}/{total}] {loc['name']}...")
        
        df = pd.read_parquet(proc_path)
        df["time"] = pd.to_datetime(df["time"], utc=True)

        d = df.sort_values("time").reset_index(drop=True)
        meta_df, X_lags = build_X_lags(d, time_col="time")

        # Tabular files - process one target at a time to save memory
        for key, col in TAB_TARGETS.items():
            tab_all = make_tabular_lagged_from_precomputed(meta_df, X_lags, d, target_col=col)
            tab_all["time"] = pd.to_datetime(tab_all["time"], utc=True)

            for split_name, (s, e) in SPLITS.items():
                s = pd.to_datetime(s, utc=True)
                e = pd.to_datetime(e, utc=True)
                issue_end = e - pd.Timedelta(hours=HORIZON)
                mask = (tab_all["time"] >= s) & (tab_all["time"] <= issue_end)
                out = tab_all.loc[mask].reset_index(drop=True)

                out_path = TAB_DIR / f"{loc_id}_{split_name}_tab_{key}_lag{LAG}_h{HORIZON}.parquet"
                out.to_parquet(out_path, index=False, compression='snappy')
                print(f"  [ok] tabular {loc['name']}/{key}/{split_name}", out.shape)

                del out
                gc.collect()

            del tab_all
            gc.collect()

        # Sequences
        Xs, Ys, Ts, meta = make_rnn_sequences(d)
        Ts = pd.to_datetime(Ts, utc=True)

        for split_name, (s, e) in SPLITS.items():
            s = pd.to_datetime(s, utc=True)
            e = pd.to_datetime(e, utc=True)
            issue_end = e - pd.Timedelta(hours=HORIZON)

            idx = (Ts >= s) & (Ts <= issue_end)
            X_out, Y_out, T_out = Xs[idx], Ys[idx], Ts[idx].to_numpy("datetime64[ns]")

            seq_path = SEQ_DIR / f"{loc_id}_{split_name}_seq_multi_lag{LAG}_h{HORIZON}.npz"
            np.savez_compressed(seq_path, X=X_out, Y=Y_out, T=T_out, meta=json.dumps(meta))
            print(f"  [ok] sequences {loc['name']}/{split_name}", X_out.shape, Y_out.shape)

        del df, d, meta_df, X_lags, Xs, Ys, Ts
        gc.collect()

# -------------------------
# RUN
# -------------------------
fetch_full_to_raw()
process_full_raw_to_processed()
build_all_outputs()

print("\n" + "="*60)
print("✅ DONE!")
print("="*60)
print(f"Total locations: {len(LOCATIONS)}")
print(f"Tabular dir: {TAB_DIR}")
print(f"Sequences dir: {SEQ_DIR}")
print(f"Location metadata: {meta_path}")
print(f"LAG (lookback): {LAG}h")
print(f"HORIZON (forecast): {HORIZON}h ({HORIZON//24} days)")
print("\nLocation IDs:")
for loc in LOCATIONS:
    print(f"  {loc['name']:15s} = {loc['location_id']}")

## ✅ Sau khi chạy xong: Upload thành Kaggle Dataset

**Output structure (20 tỉnh/thành):**
```
/kaggle/working/weather_20loc/data/
├── meta/
│   └── locations.json       <- mapping location_id -> name, lat, lon (20 locations)
├── tabular/
│   └── {location_id}_{split}_tab_{target}_lag49_h100.parquet
│       (20 locations × 7 targets × 3 splits = 420 files)
└── sequences/
    └── {location_id}_{split}_seq_multi_lag49_h100.npz
        (20 locations × 3 splits = 60 files)
```

**Danh sách 34 tỉnh/thành (theo quy hoạch sáp nhập):**
- **Miền Bắc (15):** Hà Nội, Quảng Ninh, Cao Bằng, Lạng Sơn, Lai Châu, Điện Biên, Sơn La, Tuyên Quang (+HG), Lào Cai (+YB), Thái Nguyên (+BK), Phú Thọ (+HB,VP), Bắc Ninh (+BG), Hưng Yên (+TB), Hải Phòng (+HD), Ninh Bình (+HN,NĐ)
- **Miền Trung (11):** Thanh Hóa, Nghệ An, Hà Tĩnh, Quảng Trị (+QB), Huế, Đà Nẵng (+QN), Quảng Ngãi (+KT), Gia Lai (+BĐ), Khánh Hòa (+NT), Đắk Lắk (+PY), Lâm Đồng (+ĐN,BT)
- **Miền Nam (8):** TPHCM (+BV,BD), Đồng Nai (+BP), Tây Ninh (+LA), Cần Thơ (+ST,HG), Vĩnh Long (+BT,TV), Đồng Tháp (+TG), Cà Mau (+BL), An Giang (+KG)

**Thay đổi so với phiên bản 63 tỉnh:**
- Giảm từ 63 xuống 20 locations để tránh rate limit API
- Tăng delay giữa requests lên 2 giây
- Cải thiện xử lý 429 errors với exponential backoff

**Bước tiếp theo:**
1. Chạy notebook trên Kaggle (~15-20 phút để fetch)
2. Download folder `weather_20loc/` từ Output
3. Upload lên Kaggle Dataset
4. Add dataset vào training notebooks (GRU, TCN, XGB, LGB, Ridge)